<a href="https://colab.research.google.com/github/yshun2272/stock-dashboard/blob/main/230707_Stock_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dash
!pip install dash-bootstrap-components

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 21.7 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.3.6
    Uninstalling Werkzeug-2.3.6:
      Successfully uninstalled Werkzeug-2.3.6
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.6/220.6 kB 4.0 MB/s eta 0:00:00


In [ ]:
import yfinance as yf
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

# Define the ticker symbols
tickers = ['TSLA', 'RIVN', 'NIO', 'BYDDF', 'XPEV', 'EVGO', 'LI', 'ON', 'AEHR', 'CHPT', 'BLNK', 'QS', 'MGA']

# Function to fetch stock price history for a given ticker
def get_stock_price(ticker):
    stock = yf.Ticker(ticker)
    history = stock.history(period="max")

    # Calculate 50-day and 200-day moving averages
    history['MA50'] = history['Close'].rolling(window=50).mean()
    history['MA200'] = history['Close'].rolling(window=200).mean()

    return history

# Fetch stock price history for each ticker
data = {ticker: get_stock_price(ticker) for ticker in tickers}

# Create the Dash app
app = dash.Dash(__name__)

# Define the layout of the dashboard
app.layout = html.Div(
    [
        html.H1(children='Stock Performance Dashboard'),
        dcc.Tabs(
            id='ticker-tabs',
            value=tickers[0],
            children=[
                dcc.Tab(
                    label=ticker,
                    value=ticker
                )
                for ticker in tickers
            ]
        ),
        html.Div(id='graph-content')
    ]
)

# Callback to update the graph content based on the selected ticker
@app.callback(Output('graph-content', 'children'), [Input('ticker-tabs', 'value')])
def update_graph(active_tab):
    if not active_tab:
        return []

    price_data = data.get(active_tab)

    if price_data is None:
        return []

    price_data = price_data.tail(30)

    fig = go.Figure()

    # Add black line for stock price history
    fig.add_trace(go.Scatter(x=price_data.index, y=price_data['Close'], mode='lines', name=active_tab, line=dict(color='black')))

    # Add dashed blue line for MA50
    fig.add_trace(go.Scatter(x=price_data.index, y=price_data['MA50'], mode='lines', name='MA50', line=dict(color='blue', dash='dash')))

    # Add dashed green line for MA200
    fig.add_trace(go.Scatter(x=price_data.index, y=price_data['MA200'], mode='lines', name='MA200', line=dict(color='green', dash='dash')))

    # Add markers for golden cross
    golden_cross_points = price_data[price_data['MA50'] > price_data['MA200']].index
    fig.add_trace(go.Scatter(x=golden_cross_points, y=price_data.loc[golden_cross_points]['Close'],
                             mode='markers', name='Golden Cross', marker=dict(color='gold', size=8)))

    fig.update_layout(
        title=f'Stock Price - {active_tab}',
        xaxis={'title': 'Date'},
        yaxis={'title': 'Price'}
    )

    return dcc.Graph(figure=fig)

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>